In [1]:
# %pip list

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_circles, make_moons
from sklearn.model_selection import train_test_split
import numpy as np
import plotly.express as px
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchsummary

## Craete a simple MLP for a synthetic data set

### Try Sklearn Circles dataset

In [ ]:
# create synthetic data
X,y = make_circles(n_samples=2000, shuffle=True, noise=0.075, random_state=2024, factor=0.5)
print(f"{X.shape=}, {y.shape=}")
x_train,x_test, y_train,  y_test = train_test_split(X, y.reshape((-1,1)), test_size=0.2)
print(f"{x_train.shape=}, {x_test.shape=} {y_train.shape=}, {y_test.shape=}")

X.shape=(2000, 2), y.shape=(2000,)
x_train.shape=(1600, 2), x_test.shape=(400, 2) y_train.shape=(1600, 1), y_test.shape=(400, 1)


In [ ]:
px.scatter(x = X[:,0], y = X[:,1], color = y, color_continuous_scale="geyser")

### Create pytorch data loaders

In [ ]:
x_train,x_test, y_train,  y_test =  map(lambda a : torch.tensor(a).to(torch.float32), (x_train,x_test, y_train,  y_test))
train_data = TensorDataset(x_train, y_train)
train_dataloader = DataLoader(train_data, batch_size=32)

test_data = TensorDataset(x_test.to(torch.float32), y_test.to(torch.float32))
test_dataloader = DataLoader(test_data, batch_size=64)

### Create model

In [ ]:
class MyMLPModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10, 16),
            nn.ReLU(),
            nn.Linear(16, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logit = self.mlp(x)
        return logit

mlp_model = MyMLPModel()
torchsummary.summary(mlp_model, input_size=(2,))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              30
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 16]             176
              ReLU-4                   [-1, 16]               0
            Linear-5                   [-1, 10]             170
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 1]              11
           Sigmoid-8                    [-1, 1]               0
Total params: 387
Trainable params: 387
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


### Now train the model

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=1e-2)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred[pred >= 0.5] = 1.0
            pred[pred < 0.5] = 0
            correct += ( pred == y).sum().item()
    test_loss /= num_batches
    # correct /= size
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f?} \n")
    print(f"Test Error: \n Accuracy: {correct}/{size}, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, mlp_model, loss_fn, optimizer)
    test(test_dataloader, mlp_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.253787  [   32/ 1600]
loss: 0.241740  [  352/ 1600]
loss: 0.222159  [  672/ 1600]
loss: 0.200955  [  992/ 1600]
loss: 0.133910  [ 1312/ 1600]
Test Error: 
 Accuracy: 399/400, Avg loss: 0.068007 

Epoch 2
-------------------------------
loss: 0.070496  [   32/ 1600]
loss: 0.033545  [  352/ 1600]
loss: 0.007335  [  672/ 1600]
loss: 0.002858  [  992/ 1600]
loss: 0.010094  [ 1312/ 1600]
Test Error: 
 Accuracy: 400/400, Avg loss: 0.004362 

Epoch 3
-------------------------------
loss: 0.009743  [   32/ 1600]
loss: 0.001326  [  352/ 1600]
loss: 0.000539  [  672/ 1600]
loss: 0.000361  [  992/ 1600]
loss: 0.011650  [ 1312/ 1600]
Test Error: 
 Accuracy: 399/400, Avg loss: 0.002715 

Epoch 4
-------------------------------
loss: 0.000761  [   32/ 1600]
loss: 0.000665  [  352/ 1600]
loss: 0.000137  [  672/ 1600]
loss: 0.000386  [  992/ 1600]
loss: 0.005913  [ 1312/ 1600]
Test Error: 
 Accuracy: 399/400, Avg loss: 0.001950 

Epoch 5
----------------

### Test the model

In [ ]:
pred = mlp_model(x_test)
pred[pred>=0.5] = 1.0
pred[pred <0.5] = 0.0
(pred == y_test).sum().item()
# _y.sum().item()

400